In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense,LSTM,RepeatVector, TimeDistributed,Dropout
from keras.models import Sequential,Model
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras import backend as K

In [0]:
monthly=pd.read_csv('./monthly_series.csv')
labels=pd.read_csv('./labels-series.csv',names=['index','vals'])

In [0]:
monthly.head()

,Unnamed: 0,shop_id,item_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,0,2,30,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,31,0,4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,2,32,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,2,2,0,2,0,0,1,0,0,0,0,1,0,0
3,3,2,33,1,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,1,0,1,0
4,4,2,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
#monthly=monthly.drop('Unnamed: 0',axis=1)
labels=labels.drop('index',axis=1)

In [0]:
data=pd.read_csv('./data-series.csv')


In [0]:
data=data.drop('Unnamed: 0',axis=1)

In [0]:
labels.shape

(222586, 1)

In [0]:
from lightgbm import LGBMRegressor
model=LGBMRegressor(
        n_estimators=200,
        learning_rate=0.03,
        num_leaves=32,
        colsample_bytree=0.9497036,
        subsample=0.8715623,
        max_depth=8,
        reg_alpha=0.04,
        reg_lambda=0.073,
        min_split_gain=0.0222415,
        min_child_weight=40)
model.fit(data, labels)


LGBMRegressor(boosting_type='gbdt', class_weight=None,
              colsample_bytree=0.9497036, importance_type='split',
              learning_rate=0.03, max_depth=8, min_child_samples=20,
              min_child_weight=40, min_split_gain=0.0222415, n_estimators=200,
              n_jobs=-1, num_leaves=32, objective=None, random_state=None,
              reg_alpha=0.04, reg_lambda=0.073, silent=True,
              subsample=0.8715623, subsample_for_bin=200000, subsample_freq=0)

In [0]:
model1=LGBMRegressor(learning_rate=0.05,
        metric='rmse',
        verbose= 0,
        random_state=2018,colsample_bytree=0.8, max_depth=8, n_estimators=100, num_leaves=15, subsample=1.0)
#colsample_bytree=0.8, max_depth=8, n_estimators=100, num_leaves=15, subsample=1.0
model1.fit(data,labels)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
              importance_type='split', learning_rate=0.05, max_depth=8,
              metric='rmse', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=15,
              objective=None, random_state=2018, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0, verbose=0)

In [0]:
monthly_series=monthly
test=pd.read_csv('./test.csv')
latest_records = monthly_series.drop_duplicates(subset=['shop_id', 'item_id'])
X_test = pd.merge(test, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])
X_test.fillna(0, inplace=True)
X_test.drop(['ID', 'item_id', 'shop_id'], axis=1, inplace=True)
X_test.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,7131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,7144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
cc=latest_records.drop('Unnamed: 0',axis=1)

In [0]:
labels.shape

(222586, 1)

In [0]:
serie_size=12

In [0]:
X_test = X_test[['{}'.format(i + (34 - serie_size)) for i in range(serie_size)]]
X_test.head()

,22,23,24,25,26,27,28,29,30,31,32,33
0,1.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
X_test.shape

(214200, 12)

In [0]:
pred=model.predict(X_test)


In [0]:
pred1=model1.predict(X_test)

In [0]:
mlp_prediction = pd.DataFrame(test['ID'], columns=['ID'])
mlp_prediction['item_cnt_month'] = pred1.clip(0., 20.)
mlp_prediction.to_csv('predslgb3.csv', index=False)
mlp_prediction.head()

,ID,item_cnt_month
0,0,0.450434
1,1,0.338147
2,2,0.786642
3,3,0.325745
4,4,0.338147


In [0]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
 
from sklearn.datasets import load_boston
from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     GridSearchCV, RandomizedSearchCV)
from sklearn.metrics import r2_score
 
from lightgbm.sklearn import LGBMRegressor
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1)

In [0]:
hyper_space = {'n_estimators': [100,500, 1000,1500],
               'max_depth':  [4, 5, 8,],
               'num_leaves': [15, 31, 63, 127],
               'subsample': [0.6, 0.7, 0.8, 1.0],
               'colsample_bytree': [0.6, 0.7, 0.8, 1.0]}


In [0]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [0]:
from sklearn.metrics import fbeta_score, make_scorer
score = make_scorer(rmse, greater_is_better=False)

In [0]:
est = LGBMRegressor(boosting='gbdt', n_jobs=-1, random_state=2018)

gs = GridSearchCV(est, hyper_space, scoring='neg_mean_squared_error', cv=4, verbose=20)
gs_results = gs.fit(X_train, y_train)
print("BEST PARAMETERS: " + str(gs_results.best_params_))
print("BEST CV SCORE: " + str(gs_results.best_score_))


Fitting 4 folds for each of 768 candidates, totalling 3072 fits
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=15, subsample=0.6 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=15, subsample=0.6, score=-1.208, total=   0.8s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=15, subsample=0.6 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=15, subsample=0.6, score=-1.174, total=   0.8s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=15, subsample=0.6 


...

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:   16.3s remaining:    0.0s


[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.6, score=-1.191, total=   0.9s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.6 


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:   17.3s remaining:    0.0s


[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.6, score=-1.168, total=   0.9s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7 
[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7, score=-1.206, total=   0.9s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7 
[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7, score=-1.173, total=   0.9s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7 
[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7, score=-1.191, total=   0.9s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7 
[CV]  colsample_bytree=0.6, max_depth=4, n_estimators=100, num_leaves=31, subsample=0.7, score=-1.168, total=   0.9s
[CV] colsample_bytree=0.6, max_depth=4, n_estimators=100, num_l

KeyboardInterrupt: ignored

In [0]:
print("BEST PARAMETERS: " + str(gs_results.best_params_))
print("BEST CV SCORE: " + str(gs_results.best_score_))
#colsample_bytree=0.8, max_depth=8, n_estimators=100, num_leaves=31, subsample=0.6, score=-1.170,
# colsample_bytree=0.8, max_depth=8, n_estimators=100, num_leaves=15, subsample=1.0, score=-1.165
#

NameError: ignored

In [0]:
from xgboost import XGBRegressor
reg=XGBRegressor(max_depth=8,
    n_estimators=10000,
    min_child_weight=500, 
    colsample_bytree=0.9, 
    subsample=1.0, 
    eta=0.5,    
    seed=201)
reg.fit(X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_test, y_test)], 
    verbose=True, 
    early_stopping_rounds = 40)

[20:46:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.2278	validation_1-rmse:1.18382
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:1.21173	validation_1-rmse:1.16994
[2]	validation_0-rmse:1.19125	validation_1-rmse:1.15068
[3]	validation_0-rmse:1.17404	validation_1-rmse:1.13484
[4]	validation_0-rmse:1.15932	validation_1-rmse:1.12145
[5]	validation_0-rmse:1.14808	validation_1-rmse:1.11048
[6]	validation_0-rmse:1.13779	validation_1-rmse:1.10157
[7]	validation_0-rmse:1.13007	validation_1-rmse:1.09395
[8]	validation_0-rmse:1.12277	validation_1-rmse:1.08773
[9]	validation_0-rmse:1.11806	validation_1-rmse:1.0838
[10]	validation_0-rmse:1.11461	validation_1-rmse:1.08062
[11]	validation_0-rmse:1.10977	validation_1-rmse:1.07628
[12]	validation_0-rmse:1.10676	validation_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eta=0.5, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=500, missing=None,
             n_estimators=10000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=201, silent=None, subsample=1.0, verbosity=1)

In [0]:
pd.read_csv('./lgb_submission.csv')

,"ID,item_cnt_month"
0,"0,0.9275447655520697"
1,"1,0.35250507363388184"
2,"2,1.0863763627031204"
3,"3,0.5340844298208857"
4,"4,3.2725437323734297"
5,"5,0.8890880099528599"
6,"6,0.9243659117631698"
7,"7,0.1431445942263945"
8,"8,0.8680506853929627"
9,"9,0.707211004997724"
